NLP Practical 3

In [2]:
import numpy as np
import os
from random import shuffle
import re
import sklearn as sk
import matplotlib.pyplot as plt
import random

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
import tensorflow as tf

In [5]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
raw_text = doc.xpath('//content/text()')
raw_label = doc.xpath('//head/keywords/text()')

del doc

In [6]:
talk_sentences = []
talknum = len(raw_text)

for i in range(talknum):
    temp = re.sub(r'\([^)]*\)', '', raw_text[i])
    temp = re.sub(r'\n', '', raw_text[i])
    temp = temp.split('.')
    talk_sentences.append(temp)

In [7]:
print(talk_sentences[0][1])

To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation


In [8]:
talk_sentence_word = []

for talk in talk_sentences:
    temp = []
    for sent in talk:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower()).split()
        temp.append(tokens)#
    talk_sentence_word.append(temp)

In [9]:
talk_sentence_word[0][1]

['to',
 'me',
 'the',
 'real',
 'real',
 'solution',
 'to',
 'quality',
 'growth',
 'is',
 'figuring',
 'out',
 'the',
 'balance',
 'between',
 'two',
 'activities',
 'exploration',
 'and',
 'exploitation']

In [10]:
freq = {}

for talk in talk_sentence_word:
    for sent in talk:
        for word in sent:
            if word in freq:
                freq[word] += 1
            else:
                freq[word] = 1

In [11]:
from collections import Counter

counts_ted_top1000 = []
words_top_ted = []

for word, count in Counter(freq).most_common(1000):
    counts_ted_top1000.append(count)
    words_top_ted.append(word)

In [12]:
talk_sentence_word_nostop = talk_sentence_word

for stop in words_top_ted[:200]:
    for talk in talk_sentence_word_nostop:
        for sent in talk:
            for word in sent:
                if word == stop:
                    sent.remove(word)

In [13]:
talk_sentence_word_nostop[0][1]

['real',
 'real',
 'solution',
 'quality',
 'growth',
 'figuring',
 'balance',
 'between',
 'activities',
 'exploration',
 'exploitation']

In [14]:
np.shape(talk_sentence_word_nostop)

(2085,)

In [15]:
input_label = []

for i in range(len(raw_label)):
    temp = re.sub(r' ', '', raw_label[i])
    input_label.append(temp.split(','))
    
#print(input_label_prepro)

In [16]:
labels_binary = []

for i in range(len(input_label)):
    temp = np.zeros(3)
    if 'technology' in '~'.join(input_label[i]):
        temp[0] = 1
    if 'entertainment' in '~'.join(input_label[i]):
        temp[1] = 1
    if 'design' in '~'.join(input_label[i]):
        temp[2] = 1
    labels_binary.append(temp)
        
print(input_label[9])
print(labels_binary[:10])

['talks', 'Senses', 'augmentedreality', 'brain', 'computers', 'creativity', 'cyborg', 'demo', 'design', 'engineering', 'entrepreneur', 'innovation', 'interfacedesign', 'invention', 'neuroscience', 'potential', 'prediction', 'productdesign', 'technology', 'visualizations']
[array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  1.]), array([ 1.,  0.,  1.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 1.,  0.,  1.])]


In [17]:
temp = list(zip(talk_sentence_word_nostop, labels_binary))
random.shuffle(temp)
talk_sentence_word_nostop_shuffle, labels_binary_shuffle = zip(*temp)

In [18]:
labels_onehot = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary[i][0]*4+labels_binary[i][1]*2+labels_binary[i][2]*1] = 1
    labels_onehot.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [19]:
labels_onehot[:10]

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])]

In [20]:
labels_onehot_shuffle = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary_shuffle[i][0]*4+labels_binary_shuffle[i][1]*2+labels_binary_shuffle[i][2]*1] = 1
    labels_onehot_shuffle.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [21]:
train_data = talk_sentence_word_nostop[:1835]
test_data = talk_sentence_word_nostop[1835:]

train_labels = labels_binary[:1835]
test_labels = labels_binary[1835:]

train_labels_onehot = labels_onehot[:1835]
test_labels_onehot = labels_onehot[1835:]

In [22]:
train_data_shuffle = talk_sentence_word_nostop_shuffle[:1835]
test_data_shuffle = talk_sentence_word_nostop_shuffle[1835:]

train_labels_shuffle = labels_binary_shuffle[:1835]
test_labels_shuffle = labels_binary_shuffle[1835:]

train_labels_onehot_shuffle = labels_onehot_shuffle[:1835]
test_labels_onehot_shuffle = labels_onehot_shuffle[1835:]

In [23]:
print(np.shape(talk_sentence_word_nostop))
print(np.shape(labels_binary))
print(np.shape(labels_onehot))
print(np.shape(train_data))
print(np.shape(train_labels))
print(np.shape(train_labels_onehot))
print(np.shape(test_data))
print(np.shape(test_labels))
print(np.shape(test_labels_onehot))

(2085,)
(2085, 3)
(2085, 8)
(1835,)
(1835, 3)
(1835, 8)
(250,)
(250, 3)
(250, 8)


In [24]:
print(np.shape(talk_sentence_word_nostop_shuffle))
print(np.shape(labels_binary_shuffle))
print(np.shape(labels_onehot_shuffle))
print(np.shape(train_data_shuffle))
print(np.shape(train_labels_shuffle))
print(np.shape(train_labels_onehot_shuffle))
print(np.shape(test_data_shuffle))
print(np.shape(test_labels_shuffle))
print(np.shape(test_labels_onehot_shuffle))

(2085,)
(2085, 3)
(2085, 8)
(1835,)
(1835, 3)
(1835, 8)
(250,)
(250, 3)
(250, 8)


In [25]:
check = np.asarray(train_labels_onehot)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot)
print(check.sum(0) / check.sum())

[ 0.57493188  0.0746594   0.0719346   0.00980926  0.17711172  0.06376022
  0.01525886  0.01253406]
[ 0.292  0.108  0.164  0.024  0.224  0.112  0.032  0.044]


In [26]:
check = np.asarray(train_labels_onehot_shuffle)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot_shuffle)
print(check.sum(0) / check.sum())

[ 0.53950954  0.08010899  0.08119891  0.01253406  0.18256131  0.06975477
  0.01689373  0.01743869]
[ 0.552  0.068  0.096  0.004  0.184  0.068  0.02   0.008]


In [27]:
train_lenghts = []

for talk in train_data_shuffle:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    train_lenghts.append(N)

In [28]:
train_lenghts[:10]

[824, 1389, 960, 1130, 864, 592, 1335, 579, 795, 334]

In [29]:
test_lenghts = []

for talk in test_data_shuffle:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    test_lenghts.append(N)

In [30]:
test_lenghts[:10]

[310, 734, 290, 759, 812, 720, 969, 438, 74, 1001]

In [31]:
train_talk_word = []

for talk in train_data_shuffle:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    train_talk_word.append(temp)
    
print(len(train_talk_word[0]))

824


In [32]:
test_talk_word = []

for talk in test_data_shuffle:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    test_talk_word.append(temp)
    
print(len(test_talk_word[0]))

310


In [33]:
test_talk_word[0][:10]

['quite',
 'honor',
 'tonight',
 'glad',
 'stayed',
 'listened',
 'inspired',
 'play',
 'songs',
 'tonight']

In [34]:
train_data[0][:5]

[['reasons', 'companies', 'fail'],
 ['real',
  'real',
  'solution',
  'quality',
  'growth',
  'figuring',
  'balance',
  'between',
  'activities',
  'exploration',
  'exploitation'],
 ['both', 'necessary', 'too'],
 ['consider', 'facit'],
 ['old', 'enough', 'remember']]

In [35]:
vocab = {}
N = 1

for talk in train_data_shuffle:
    for sent in talk:
        for word in sent:
            if word in vocab:
                N = N
            else:
                vocab[word] = N
                N += 1
                
vocab["unknown_word"] = 0

In [36]:
vocab["unknown_word"]

0

In [37]:
train_talk_word_index = []

for talk in train_talk_word:
    temp = []
    for word in talk:
        if word in vocab:
            temp.append(vocab[word])
        else:
            temp.append(vocab["unknown_word"])
    train_talk_word_index.append(temp)

In [38]:
test_talk_word_index = []

for talk in test_talk_word:
    temp = []
    for word in talk:
        if word in vocab:
            temp.append(vocab[word])
        else:
            temp.append(vocab["unknown_word"])
    test_talk_word_index.append(temp)

In [39]:
train_talk_word_index[0][:20]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 3, 7, 11, 12, 13, 14, 15, 2, 16, 17]

In [40]:
train_talk_word_index_exp = []

for i in range(len(train_talk_word_index)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (train_lenghts[i]-1):
            temp.append(train_talk_word_index[i][j])
        else:
            temp.append(0)
    train_talk_word_index_exp.append(temp)

In [41]:
test_talk_word_index_exp = []

for i in range(len(test_talk_word_index)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (test_lenghts[i]-1):
            temp.append(test_talk_word_index[i][j])
        else:
            temp.append(0)
    test_talk_word_index_exp.append(temp)

In [42]:
np.shape(test_talk_word_index_exp)

(250, 2221)

In [43]:
np.shape(train_talk_word_index_exp)

(1835, 2221)

In [44]:
test_talk_word_index_exp[0]

[87,
 1399,
 4755,
 5205,
 1577,
 7011,
 5036,
 371,
 5634,
 4755,
 402,
 20488,
 366,
 2002,
 2848,
 5634,
 1501,
 1640,
 5935,
 1962,
 1623,
 4109,
 356,
 186,
 4182,
 3812,
 1668,
 1520,
 3626,
 7119,
 1355,
 712,
 1899,
 582,
 8499,
 8874,
 766,
 5093,
 0,
 1032,
 5399,
 1520,
 1557,
 1962,
 1956,
 4769,
 2495,
 1180,
 8399,
 1368,
 1229,
 1897,
 999,
 5580,
 12709,
 1009,
 18,
 1205,
 2081,
 846,
 6031,
 813,
 5116,
 146,
 4769,
 1909,
 4769,
 274,
 146,
 4769,
 47442,
 1429,
 1042,
 812,
 449,
 2598,
 1060,
 1060,
 1467,
 4930,
 10051,
 4416,
 161,
 0,
 10915,
 3363,
 340,
 3677,
 1654,
 15286,
 0,
 2081,
 63,
 28377,
 624,
 1719,
 146,
 4769,
 1909,
 4769,
 274,
 146,
 4769,
 14048,
 20051,
 624,
 1042,
 6070,
 3544,
 16721,
 1060,
 14512,
 2775,
 962,
 1704,
 542,
 3369,
 16721,
 7351,
 54,
 962,
 1002,
 18775,
 2581,
 123,
 1893,
 133,
 241,
 552,
 1207,
 11977,
 19888,
 48165,
 4483,
 230,
 146,
 4769,
 2557,
 4769,
 3369,
 778,
 1934,
 7488,
 4769,
 4769,
 2598,
 3369,
 1166

In [45]:
train_labels_RNN = np.argmax(train_labels_onehot_shuffle, 1) 

In [46]:
train_labels_RNN[:10]

array([0, 4, 0, 2, 4, 7, 4, 4, 0, 5])

In [47]:
test_labels_RNN = np.argmax(test_labels_onehot_shuffle, 1) 

In [48]:
class SimpleDataIterator():
    def __init__(self, texts, lengths, labels):
        self.texts = texts
        self.lengths = lengths
        self.labels = labels
        self.size = len(self.texts)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        temp = list(zip(self.texts, self.lengths, self.labels))
        random.shuffle(temp)
        self.texts, self.lengths, self.labels = zip(*temp)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res_texts = self.texts[self.cursor:self.cursor+n]
        res_lengths = self.lengths[self.cursor:self.cursor+n]
        res_labels = self.labels[self.cursor:self.cursor+n]
        self.cursor += n
        return res_texts, res_labels, res_lengths

In [49]:
data = SimpleDataIterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')
print('Target values\n', d[1], end='\n\n')
print('Sequence lengths\n', d[2])

Input sequences
 ([2576, 22480, 22481, 2576, 5533, 1541, 83, 3182, 1984, 387, 13459, 22482, 2576, 63, 5975, 17519, 236, 1887, 1292, 497, 3427, 1129, 2576, 1856, 2190, 9926, 34, 22483, 565, 22484, 12571, 2097, 2263, 11367, 2843, 4799, 264, 4799, 453, 6138, 926, 966, 22485, 18388, 21912, 986, 3189, 22486, 6170, 4203, 22487, 864, 3427, 2576, 4799, 2266, 10565, 4357, 1769, 631, 18612, 22488, 191, 2843, 6684, 5177, 631, 2190, 2754, 6862, 3886, 3776, 690, 22482, 1336, 4452, 690, 194, 1336, 191, 10182, 690, 52, 1336, 3841, 29, 631, 191, 5177, 631, 1072, 1659, 22489, 22490, 4218, 3721, 908, 859, 2190, 7756, 387, 631, 13459, 5975, 22482, 2576, 2190, 8248, 17519, 931, 141, 5975, 64, 360, 990, 672, 2576, 990, 17519, 8248, 5965, 5440, 22491, 17519, 19644, 19644, 193, 2906, 5965, 7196, 1751, 5944, 98, 2362, 5965, 11, 803, 3758, 484, 683, 15122, 1271, 1259, 690, 9591, 1057, 15122, 630, 1439, 541, 98, 2328, 513, 3481, 8351, 9932, 22492, 5550, 16485, 5276, 1292, 1119, 1211, 672, 986, 2576, 1856, 8949,

In [50]:
np.shape(train_talk_word_index_exp)

(1835, 2221)

In [51]:
np.shape(test_talk_word_index_exp)

(250, 2221)

In [52]:
np.shape(train_lenghts)

(1835,)

In [53]:
np.shape(test_lenghts)

(250,)

In [54]:
np.shape(train_labels_RNN)

(1835,)

In [55]:
np.shape(test_labels_RNN)

(250,)

In [56]:
len(vocab)

52392

In [59]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(vocab)+1,
    state_size = 100,
    batch_size = 50,
    num_classes = 8):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.constant(1.0)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.LSTMCell(state_size)
    init_state = tf.get_variable('init_state', [1, 2*state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                 initial_state=init_state, dtype=tf.float32)

    # Add dropout, as the model otherwise quickly overfits
    #rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    #idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    #last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)
    
    last_rnn_output = tf.reduce_mean(rnn_outputs, 1)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

def train_graph(graph, batch_size = 50, num_epochs = 10, iterator = SimpleDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
        te = iterator(test_talk_word_index_exp, test_lenghts, test_labels_RNN)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [112]:
g = build_graph(vocab_size=len(vocab)+1, batch_size=50, num_classes=8, state_size=100)

In [113]:
tr_losses, te_losses = train_graph(g, batch_size = 50, num_epochs = 10)

Accuracy after epoch 1  - tr: 0.52486486451 - te: 0.606666674217
Accuracy after epoch 2  - tr: 0.531111112071 - te: 0.592000007629
Accuracy after epoch 3  - tr: 0.541111112469 - te: 0.584000003338
Accuracy after epoch 4  - tr: 0.533333333002 - te: 0.584000015259
Accuracy after epoch 5  - tr: 0.532222220467 - te: 0.572000008821
Accuracy after epoch 6  - tr: 0.534444444709 - te: 0.608000016212
Accuracy after epoch 7  - tr: 0.532777777149 - te: 0.579999995232
Accuracy after epoch 8  - tr: 0.534999999735 - te: 0.596000003815
Accuracy after epoch 9  - tr: 0.533888889684 - te: 0.604000008106
Accuracy after epoch 10  - tr: 0.534444443882 - te: 0.584000009298


In [119]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(vocab)+1,
    state_size = 100,
    batch_size = 50,
    num_classes = 8):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.constant(1.0)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    fw_cell = tf.contrib.rnn.LSTMCell(state_size)
    bw_cell = tf.contrib.rnn.LSTMCell(state_size)
    init_state = tf.get_variable('init_state', [1, 2*state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.bidirectional_dynamic_rnn(fw_cell, bw_cell, rnn_inputs, sequence_length=seqlen,
                                                      dtype=tf.float32)
    
    # Add dropout, as the model otherwise quickly overfits
    #rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    #idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    #last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)
    
    print(np.shape(rnn_outputs[0]))
    
    last_rnn_output = tf.concat([rnn_outputs[0], rnn_outputs[1]], 2)
    
    
    last_rnn_output = tf.reduce_mean(last_rnn_output, 1)
    print(np.shape(last_rnn_output))


    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [2*state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }
    
def train_graph(graph, batch_size = 50, num_epochs = 10, iterator = SimpleDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
        te = iterator(test_talk_word_index_exp, test_lenghts, test_labels_RNN)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [120]:
g = build_graph(vocab_size=len(vocab)+1, batch_size=50, num_classes=8, state_size=100)

(50, ?, 100)
(50, 200)


In [121]:
tr_losses, te_losses = train_graph(g, batch_size = 50, num_epochs = 10)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Accuracy after epoch 1  - tr: 0.528108105265 - te: 0.540000006557
Accuracy after epoch 2  - tr: 0.536666662329 - te: 0.575999999046
Accuracy after epoch 3  - tr: 0.539444445736 - te: 0.535999983549
Accuracy after epoch 4  - tr: 0.536111114754 - te: 0.551999998093
Accuracy after epoch 5  - tr: 0.54222222252 - te: 0.571999996901
Accuracy after epoch 6  - tr: 0.537222224805 - te: 0.536000007391
Accuracy after epoch 7  - tr: 0.541111109157 - te: 0.583999991417
Accuracy after epoch 8  - tr: 0.541111110813 - te: 0.515999996662
Accuracy after epoch 9  - tr: 0.536666663984 - te: 0.551999998093
Accuracy after epoch 10  - tr: 0.543888889253 - te: 0.560000008345


RNNs have an advantage because for the words are inputted in a sequence therefore there is a possiblity that the NN can actually infer something from the sequence rather than just the bag of words representation which completelly ignores order and context. However, this method takes significantly longer and is even more suceptable to data availablility, because contex is now something else that needs to learned and it requires extra data.

Using just the final state as the word representation actually started off better than bag of words but ended up just predicting ooo. Dropout seems to be the biggest help.

After trying different architectures (LSTM, GRU, Standard RNN cell) no significant differences were found between the model accuracies. We blame it on the data once more.

Biderectional LSTM also didnt seem to help much and in fact did worse